In [1]:
from ase.io import Trajectory
import torch
torch.manual_seed(0)
import numpy as np
from nnmd.features import calculate_sf, calculate_params

In [2]:
traj = Trajectory('input/Li_crystal_27.traj')
positions = np.array([atoms.positions for atoms in traj[1000:1002]])
cell = torch.tensor(traj[1].cell.array, dtype = torch.float32, device = 'cuda')
cartesians = torch.tensor(positions, dtype = torch.float32, device = 'cuda')
cartesians.requires_grad = True

In [3]:
r_cutoff = 38.0
N_g1, N_g2, N_g3, N_g4, N_g5 = 0, 25, 0, 25, 0

features = [1] * N_g1 + [2] * N_g2 + [3] * N_g3 + [4] * N_g4 + [5] * N_g5
params = calculate_params(r_cutoff, N_g1, N_g2, N_g3, N_g4, N_g5)
symm_funcs_data = {'features': features, 'params': params}

g, dg = calculate_sf(cartesians, cell, symm_funcs_data)
print('g.shape:', g.shape)
g

Calculating symmetry functions:   0%|          | 0/1 [00:00<?, ?it/s]/home/andrey-win/anaconda3/envs/test-nnmd/lib/python3.11/site-packages/torch/autograd/graph.py:769: UserWarning: Error detected in MulBackward0. Traceback of forward call that caused the error:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/andrey-win/anaconda3/envs/test-nnmd/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/andrey-win/anaconda3/envs/test-nnmd/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/andrey-win/anaconda3/envs/test-nnmd/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/andrey-win/anaconda3/envs/test-nnmd/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File 

RuntimeError: Function 'MulBackward0' returned nan values in its 1th output.

In [4]:
(g.max(dim = 1, keepdim = True)[0] - g.min(dim = 1, keepdim = True)[0]).unsqueeze(-1).expand_as(dg).size()

RuntimeError: expand(torch.cuda.FloatTensor{[1, 1, 27, 50, 1]}, size=[1, 27, 50, 3]): the number of sizes provided (4) must be greater or equal to the number of dimensions in the tensor (5)

In [5]:
torch.norm(g, p = 2, dim = 2, keepdim = True)

tensor([[[[nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan]],

         ...,

         [[nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan]],

         [[nan, nan, nan,  ..., nan, nan, nan]]]], device='cuda:0')

In [7]:
g_clamp = g / torch.norm(g, p = 2, dim = -1, keepdim = True)
for _g in g_clamp[0]:
    print(_g.cpu().numpy())

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan n